In [11]:
import torch
import torch.nn as nn
import torchvision.models as models

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from datetime import datetime

In [12]:
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]

IMAGE_SIZE = 224
BATCH_SIZE = 16

NUM_WORKERS = 2
NUM_CLASSES = 9
NUM_EPOCHS = 30
LEARNING_RATE = 1e-3

In [13]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
])

val_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
])

train_dataset_path = "../../../dataset/train"
val_dataset_path = "../../../dataset/val"

train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_dataset_path, transform=val_transforms)

train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    pin_memory=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

val_loader = DataLoader(
    val_dataset,
    shuffle=False,
    pin_memory=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [14]:
if __name__ == "__main__":
    images, labels = next(iter(train_loader))
    print(f"Batch shape: {images.shape}")
    print(f"Label shape: {labels.shape}")

Batch shape: torch.Size([16, 3, 224, 224])
Label shape: torch.Size([16])


In [15]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    training_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        training_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    
    avg_loss = training_loss/total
    avg_accuracy = correct/total

    return avg_loss, avg_accuracy

In [16]:
def validate(model, loader, criterion, device):
    model.eval()
    validate_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            validate_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    
    avg_loss = validate_loss/total
    avg_accuracy = correct/total

    return avg_loss, avg_accuracy

In [17]:
model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)

for param in model.features.parameters():
    param.requires_grad = False

old_linear = model.classifier[3]
in_feats = old_linear.in_features

classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_feats, NUM_CLASSES),
)
model.classifier[3] = classifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)

In [19]:
best_val_loss = float('inf')

for epoch in range (NUM_EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validate_loss, val_acc = validate(model, val_loader, criterion, device)

    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS}: "
        f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
        f"Val loss {validate_loss:.4f}, acc {val_acc:.4f}"
    )

    # checkpoint best model
    if validate_loss < best_val_loss:
        best_val_loss = validate_loss

        now = datetime.now()
        model_name = "./mobilenet/v3/" + "mobilenetv3" + "-" + now.strftime("%m%d%Y%H%M%S") 

        torch.save(model.state_dict(), model_name)

Epoch 1/30: Train loss 1.1758, acc 0.6425 | Val   loss 0.5885, acc 0.8343
Epoch 2/30: Train loss 0.6266, acc 0.7992 | Val   loss 0.4140, acc 0.8737
Epoch 3/30: Train loss 0.5190, acc 0.8256 | Val   loss 0.3738, acc 0.8626
Epoch 4/30: Train loss 0.4578, acc 0.8476 | Val   loss 0.2980, acc 0.9000
Epoch 5/30: Train loss 0.4349, acc 0.8552 | Val   loss 0.2963, acc 0.8960
Epoch 6/30: Train loss 0.3888, acc 0.8727 | Val   loss 0.2657, acc 0.9081
Epoch 7/30: Train loss 0.3776, acc 0.8707 | Val   loss 0.2673, acc 0.9091
Epoch 8/30: Train loss 0.3547, acc 0.8789 | Val   loss 0.2568, acc 0.9061
Epoch 9/30: Train loss 0.3560, acc 0.8776 | Val   loss 0.2498, acc 0.9081
Epoch 10/30: Train loss 0.3431, acc 0.8761 | Val   loss 0.2322, acc 0.9192
Epoch 11/30: Train loss 0.3242, acc 0.8887 | Val   loss 0.2279, acc 0.9152
Epoch 12/30: Train loss 0.3327, acc 0.8876 | Val   loss 0.2232, acc 0.9253
Epoch 13/30: Train loss 0.3225, acc 0.8852 | Val   loss 0.2292, acc 0.9162
Epoch 14/30: Train loss 0.3120, ac